In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from scipy import constants
import os
import glob
from matplotlib import rc

In [ ]:
from InfluxData import InfluxData
client = InfluxData()
client.set_client('155.230.28.170',8086,'sslab','1231',database='kmaeq')
device_ids = client.get_device()

In [ ]:
raw_data = {'dev_id':[],
            'Mean_PSD_x':[],
            'Mean_PSD_y':[],
            'Mean_PSD_z':[],
            'MSE_PSD_x':[],
            'MSE_PSD_y':[],
            'MSE_PSD_z':[]}
new_df = pd.DataFrame(raw_data)
new_df

In [ ]:
EXPECT_DATASET_SIZE = 60000 # 10min data
def psd_processing(dev_id:str):
    cli = InfluxData()
    cli.set_client('155.230.28.170',8086,'sslab','1231',database='kmaeq')
    rs = cli.query(f"select * from acc_data where dev_id='{dev_id}' and time >= '2019-04-11T04:20:00Z' AND time <= '2019-04-11T04:30:00Z'")
    data = cli.resultSetToDF(rs)
    if type(data['acc_data']) is list: #  it means empty data
        print('empty data')
        return dev_id
    if len(data['acc_data']) < EXPECT_DATASET_SIZE*0.9: # not enough data size
        print('not enough data size')
        return dev_id
    data = data['acc_data'] # convert dict_dataframe to dataframe
    x = data['x']*constants.g
    y = data['y']*constants.g
    z = data['z']*constants.g
    offset_x = x - np.mean(x)
    offset_y = y - np.mean(y)
    offset_z = z - np.mean(z)
    Pxx, freqs_x = plt.psd(offset_x,NFFT=200,Fs=100)
    Pxx_true = [0 for i in range(len(Pxx))]
    x_mse = mean_squared_error(Pxx,Pxx_true)
    Pyy, freqs_y = plt.psd(offset_y,NFFT=200,Fs=100)
    Pyy_true = [0 for i in range(len(Pyy))]
    y_mse = mean_squared_error(Pyy,Pyy_true)
    Pzz, freqs_z = plt.psd(offset_z,NFFT=200,Fs=100)
    Pzz_true = [0 for i in range(len(Pzz))]
    z_mse = mean_squared_error(Pzz,Pzz_true)
    row = {'dev_id':dev_id,
            'Mean_PSD_x':np.mean(Pxx),
            'Mean_PSD_y':np.mean(Pyy),
            'Mean_PSD_z':np.mean(Pzz),
            'MSE_PSD_x': x_mse,
            'MSE_PSD_y': y_mse,
            'MSE_PSD_z': z_mse
            }
    return row

In [ ]:
%%time
from concurrent.futures import ProcessPoolExecutor, as_completed, wait
pool = ProcessPoolExecutor(len(device_ids))
futures = []
for dev_id in device_ids:
    futures.append(pool.submit(psd_processing,dev_id))

err_devices = [] 

for x in as_completed(futures,timeout=120):
    if type(x.result()) is dict:
        new_df = new_df.append(x.result(),ignore_index=True)
    else:
        err_devices.append(x.result())

In [ ]:
'''
while(q.empty() is False):
    new_df = new_df.append(q.get(),ignore_index=True)
new_df
'''
new_df
new_df['Mean_PSD_x'].describe()